# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 8:30AM,257396,10,0086387976,ISDIN Corporation,Released
1,Feb 15 2023 8:30AM,257396,10,0086387978,ISDIN Corporation,Released
2,Feb 15 2023 8:30AM,257393,10,0086387016,ISDIN Corporation,Released
3,Feb 15 2023 8:30AM,257393,10,0086387014,ISDIN Corporation,Released
4,Feb 15 2023 8:30AM,257393,10,0086387012,ISDIN Corporation,Released
5,Feb 15 2023 8:30AM,257396,10,0086387969,ISDIN Corporation,Released
6,Feb 15 2023 8:30AM,257396,10,0086387973,ISDIN Corporation,Released
7,Feb 15 2023 8:30AM,257396,10,0086387975,ISDIN Corporation,Released
8,Feb 15 2023 8:30AM,257393,10,0086384807,ISDIN Corporation,Released
9,Feb 15 2023 8:30AM,257393,10,0086384802,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,257391,Released,1
53,257392,Released,5
54,257393,Released,55
55,257395,Released,2
56,257396,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257391,NaN,NaN,1.0
257392,NaN,NaN,5.0
257393,NaN,NaN,55.0
257395,NaN,NaN,2.0
257396,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257294,0.0,0.0,1.0
257295,0.0,0.0,3.0
257303,0.0,0.0,2.0
257308,0.0,15.0,2.0
257309,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257294,0,0,1
257295,0,0,3
257303,0,0,2
257308,0,15,2
257309,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257294,0,0,1
1,257295,0,0,3
2,257303,0,0,2
3,257308,0,15,2
4,257309,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257294,,,1
1,257295,,,3
2,257303,,,2
3,257308,,15,2
4,257309,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation
2,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation
56,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation
62,Feb 15 2023 8:30AM,257388,10,"Methapharm, Inc."
73,Feb 15 2023 8:30AM,257395,10,Emerginnova
75,Feb 14 2023 3:01PM,257391,22,"NBTY Global, Inc."
76,Feb 14 2023 3:01PM,257389,22,"NBTY Global, Inc."
77,Feb 14 2023 2:56PM,257387,10,"Methapharm, Inc."
79,Feb 14 2023 2:48PM,257386,22,"NBTY Global, Inc."
80,Feb 14 2023 2:18PM,257384,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,,,5
1,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,,,55
2,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,,,5
3,Feb 15 2023 8:30AM,257388,10,"Methapharm, Inc.",,,8
4,Feb 15 2023 8:30AM,257395,10,Emerginnova,,,2
5,Feb 14 2023 3:01PM,257391,22,"NBTY Global, Inc.",,,1
6,Feb 14 2023 3:01PM,257389,22,"NBTY Global, Inc.",,,1
7,Feb 14 2023 2:56PM,257387,10,"Methapharm, Inc.",,,2
8,Feb 14 2023 2:48PM,257386,22,"NBTY Global, Inc.",,,1
9,Feb 14 2023 2:18PM,257384,10,"Citieffe, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5,,
1,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55,,
2,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5,,
3,Feb 15 2023 8:30AM,257388,10,"Methapharm, Inc.",8,,
4,Feb 15 2023 8:30AM,257395,10,Emerginnova,2,,
5,Feb 14 2023 3:01PM,257391,22,"NBTY Global, Inc.",1,,
6,Feb 14 2023 3:01PM,257389,22,"NBTY Global, Inc.",1,,
7,Feb 14 2023 2:56PM,257387,10,"Methapharm, Inc.",2,,
8,Feb 14 2023 2:48PM,257386,22,"NBTY Global, Inc.",1,,
9,Feb 14 2023 2:18PM,257384,10,"Citieffe, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5,,
1,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55,,
2,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5,,
3,Feb 15 2023 8:30AM,257388,10,"Methapharm, Inc.",8,,
4,Feb 15 2023 8:30AM,257395,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5.0,NaN,NaN
1,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55.0,NaN,NaN
2,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5.0,NaN,NaN
3,Feb 15 2023 8:30AM,257388,10,"Methapharm, Inc.",8.0,NaN,NaN
4,Feb 15 2023 8:30AM,257395,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 8:30AM,257396,10,ISDIN Corporation,5.0,0.0,0.0
1,Feb 15 2023 8:30AM,257393,10,ISDIN Corporation,55.0,0.0,0.0
2,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5.0,0.0,0.0
3,Feb 15 2023 8:30AM,257388,10,"Methapharm, Inc.",8.0,0.0,0.0
4,Feb 15 2023 8:30AM,257395,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,9779476,333.0,23.0,0.0
15,514689,11.0,5.0,5.0
19,1543997,9.0,30.0,18.0
20,257336,1.0,0.0,0.0
21,257330,0.0,1.0,0.0
22,772166,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,9779476,333.0,23.0,0.0
1,15,514689,11.0,5.0,5.0
2,19,1543997,9.0,30.0,18.0
3,20,257336,1.0,0.0,0.0
4,21,257330,0.0,1.0,0.0
5,22,772166,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,333.0,23.0,0.0
1,15,11.0,5.0,5.0
2,19,9.0,30.0,18.0
3,20,1.0,0.0,0.0
4,21,0.0,1.0,0.0
5,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,333.0
1,15,Released,11.0
2,19,Released,9.0
3,20,Released,1.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,0.0,5.0,18.0,0.0,0.0,0.0
Executing,23.0,5.0,30.0,0.0,1.0,0.0
Released,333.0,11.0,9.0,1.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,0.0,5.0,18.0,0.0,0.0,0.0
1,Executing,23.0,5.0,30.0,0.0,1.0,0.0
2,Released,333.0,11.0,9.0,1.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,0.0,5.0,18.0,0.0,0.0,0.0
1,Executing,23.0,5.0,30.0,0.0,1.0,0.0
2,Released,333.0,11.0,9.0,1.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()